In [46]:
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

# hmm tagger 1
Following https://www.mygreatlearning.com/blog/pos-tagging/

Takes 1h30m to get accuracy on test set.

In [5]:
nltk.download('treebank')
nltk.download('brown')

[nltk_data] Downloading package treebank to /home/joey/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to /home/joey/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/joey/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [16]:
treebank = list(nltk.corpus.treebank.tagged_words())
tag_fd = nltk.FreqDist(tag for (word, tag) in treebank)
tag_fd.most_common()

[('NN', 13166),
 ('IN', 9857),
 ('NNP', 9410),
 ('DT', 8165),
 ('-NONE-', 6592),
 ('NNS', 6047),
 ('JJ', 5834),
 (',', 4886),
 ('.', 3874),
 ('CD', 3546),
 ('VBD', 3043),
 ('RB', 2822),
 ('VB', 2554),
 ('CC', 2265),
 ('TO', 2179),
 ('VBN', 2134),
 ('VBZ', 2125),
 ('PRP', 1716),
 ('VBG', 1460),
 ('VBP', 1321),
 ('MD', 927),
 ('POS', 824),
 ('PRP$', 766),
 ('$', 724),
 ('``', 712),
 ("''", 694),
 (':', 563),
 ('WDT', 445),
 ('JJR', 381),
 ('NNPS', 244),
 ('WP', 241),
 ('RP', 216),
 ('JJS', 182),
 ('WRB', 178),
 ('RBR', 136),
 ('-RRB-', 126),
 ('-LRB-', 120),
 ('EX', 88),
 ('RBS', 35),
 ('PDT', 27),
 ('#', 16),
 ('WP$', 14),
 ('LS', 13),
 ('FW', 4),
 ('UH', 3),
 ('SYM', 1)]

In [18]:
treebank = list(nltk.corpus.treebank.tagged_sents())

In [19]:
for sent in treebank[:2]:
    for tuple in sent:
        print(tuple)

('Pierre', 'NNP')
('Vinken', 'NNP')
(',', ',')
('61', 'CD')
('years', 'NNS')
('old', 'JJ')
(',', ',')
('will', 'MD')
('join', 'VB')
('the', 'DT')
('board', 'NN')
('as', 'IN')
('a', 'DT')
('nonexecutive', 'JJ')
('director', 'NN')
('Nov.', 'NNP')
('29', 'CD')
('.', '.')
('Mr.', 'NNP')
('Vinken', 'NNP')
('is', 'VBZ')
('chairman', 'NN')
('of', 'IN')
('Elsevier', 'NNP')
('N.V.', 'NNP')
(',', ',')
('the', 'DT')
('Dutch', 'NNP')
('publishing', 'VBG')
('group', 'NN')
('.', '.')


In [22]:
train_set,test_set =train_test_split(treebank,train_size=0.80,test_size=0.20,random_state = 101)

In [35]:
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
test_words = [tup[0] for sent in test_set for tup in sent]
test_tags = [tup[1] for sent in test_set for tup in sent]

print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [25]:
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

45
{'``', ':', 'JJR', ',', 'NNS', '-RRB-', 'JJS', 'UH', 'RBR', 'JJ', 'LS', 'RBS', 'PRP', 'WDT', '$', 'PRP$', '#', 'RB', "''", 'CD', 'MD', 'VBN', 'WP', 'VBP', 'VBG', 'CC', 'VB', '.', 'NN', 'DT', 'TO', '-LRB-', 'RP', 'VBZ', 'FW', 'IN', 'POS', 'WRB', 'WP$', 'VBD', 'NNPS', 'EX', 'NNP', 'PDT', '-NONE-'}


In [24]:
vocab = {word for word,tag in train_tagged_words}

In [26]:
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
     
    return (count_w_given_tag, count_tag)

In [27]:
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)


In [30]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

print(tags_matrix)

[[0.0000000e+00 0.0000000e+00 1.7543860e-03 ... 6.8421051e-02
  0.0000000e+00 3.3333335e-02]
 [5.1224943e-02 0.0000000e+00 4.4543431e-03 ... 1.7371938e-01
  0.0000000e+00 3.3407573e-02]
 [0.0000000e+00 6.4724921e-03 0.0000000e+00 ... 9.7087380e-03
  0.0000000e+00 1.9417476e-02]
 ...
 [5.4083287e-04 8.1124930e-03 1.3520822e-04 ... 3.7817740e-01
  0.0000000e+00 5.6787455e-03]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [2.3125843e-03 6.7450376e-03 1.7344382e-03 ... 3.9313935e-02
  0.0000000e+00 7.5929850e-02]]


In [33]:
df_tags = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [40]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(tqdm(words)):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = df_tags.loc['.', tag]
            else:
                transition_p = df_tags.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [41]:
tagged_seq = Viterbi(test_words)


Viterbi Algorithm Accuracy:  0.0


In [45]:
# accuracy
correct_idx = [i for i, j in zip(tagged_seq, test_tagged_words) if i == j] 
 
accuracy = len(correct_idx)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  87.62643621722479


In [62]:
6592/100000

0.06592